In [ ]:
# notebooks/characters.ipynb

# RPGシミュレーター - キャラクター定義
# 
# このノートブックでは、RPGシミュレーターで使用するキャラクタークラスと関連機能を定義します。
# 
# ## 目的
# - 基本的なキャラクタークラスの実装
# - プレイヤーキャラクターと敵キャラクターの実装
# - キャラクターステータスの計算メソッド
# - JSONからのキャラクターデータ読み込み機能

import json
import os
import random
from typing import Dict, List, Optional, Any

# ## 基本キャラクタークラス

class Character:
    """RPGシミュレーターの基本キャラクタークラス"""
    
    def __init__(self, name: str, hp: int, mp: int, strength: int, defense: int,
             magic: int, speed: int, level: int = 1):
        """キャラクターの初期化
        
        Args:
            name: キャラクター名
            hp: 最大HP
            mp: 最大MP
            strength: 攻撃力
            defense: 防御力
            magic: 魔法攻撃力
            speed: 素早さ
            level: レベル（デフォルトは1）
        """
        # super().__init__() の呼び出し削除
        
        # 属性を直接設定
        self.name = name
        self.max_hp = hp
        self.current_hp = hp
        self.max_mp = mp
        self.current_mp = mp
        self.strength = strength
        self.defense = defense
        self.magic = magic
        self.speed = speed
        self.level = level
        self.is_alive = True
        self.skills = []  # スキルリスト
        
    def attack(self, target: 'Character') -> int:
        """通常攻撃を実行する
        
        Args:
            target: 攻撃対象のキャラクター
            
        Returns:
            与えたダメージ量
        """
        if not self.is_alive or not target.is_alive:
            return 0
        
        # ダメージ計算（基本は攻撃力 - 防御力/2、最小1ダメージ保証）
        base_damage = max(1, self.strength - target.defense // 2)
        
        # ダメージの揺れ幅（±20%）
        variance = random.uniform(0.8, 1.2)
        damage = max(1, int(base_damage * variance))
        
        # ダメージを与える
        target.take_damage(damage)
        
        return damage
    
    def take_damage(self, damage: int) -> None:
        """ダメージを受ける
        
        Args:
            damage: 受けるダメージ量
        """
        self.current_hp = max(0, self.current_hp - damage)
        
        # HPが0になったら戦闘不能
        if self.current_hp == 0:
            self.is_alive = False
    
    def heal(self, amount: int) -> int:
        """HPを回復する
        
        Args:
            amount: 回復量
            
        Returns:
            実際の回復量
        """
        if not self.is_alive:
            return 0
        
        old_hp = self.current_hp
        self.current_hp = min(self.max_hp, self.current_hp + amount)
        return self.current_hp - old_hp
    
    def use_mp(self, amount: int) -> bool:
        """MPを消費する
        
        Args:
            amount: 消費量
            
        Returns:
            消費に成功したかどうか
        """
        if not self.is_alive or self.current_mp < amount:
            return False
        
        self.current_mp -= amount
        return True
    
    def recover_mp(self, amount: int) -> int:
        """MPを回復する
        
        Args:
            amount: 回復量
            
        Returns:
            実際の回復量
        """
        if not self.is_alive:
            return 0
        
        old_mp = self.current_mp
        self.current_mp = min(self.max_mp, self.current_mp + amount)
        return self.current_mp - old_mp
    
    def reset(self) -> None:
        """戦闘開始時の状態にリセットする"""
        self.current_hp = self.max_hp
        self.current_mp = self.max_mp
        self.is_alive = True
    
    def status(self) -> Dict:
        """キャラクターの現在の状態を取得する
        
        Returns:
            キャラクターの状態を表す辞書
        """
        return {
            'name': self.name,
            'level': self.level,
            'hp': f'{self.current_hp}/{self.max_hp}',
            'mp': f'{self.current_mp}/{self.max_mp}',
            'strength': self.strength,
            'defense': self.defense,
            'magic': self.magic,
            'speed': self.speed,
            'is_alive': self.is_alive
        }

    def learn_skill(self, skill_name: str) -> bool:
        """スキルを習得する
        
        Args:
            skill_name: 習得するスキル名
            
        Returns:
            習得に成功したかどうか
        """
        # すでに習得済みの場合は失敗
        if skill_name in self.skills:
            return False
        
        # スキルの名前を保存
        self.skills.append(skill_name)
        return True
    
    def forget_skill(self, skill_name: str) -> bool:
        """スキルを忘れる
        
        Args:
            skill_name: 忘れるスキル名
            
        Returns:
            忘れるのに成功したかどうか
        """
        if skill_name not in self.skills:
            return False
        
        self.skills.remove(skill_name)
        return True
# ## プレイヤーキャラクタークラス

class PlayerCharacter(Character):
    """プレイヤーキャラクター用のクラス"""
    
    def __init__(self, name: str, hp: int, mp: int, strength: int, defense: int,
                 magic: int, speed: int, level: int = 1, job: str = "戦士",
                 exp: int = 0, next_level_exp: int = 100):
        """プレイヤーキャラクターの初期化
        
        Args:
            name: キャラクター名
            hp: 最大HP
            mp: 最大MP
            strength: 攻撃力
            defense: 防御力
            magic: 魔法攻撃力
            speed: 素早さ
            level: レベル（デフォルトは1）
            job: 職業（デフォルトは「戦士」）
            exp: 現在の経験値（デフォルトは0）
            next_level_exp: 次のレベルに必要な経験値（デフォルトは100）
        """
        super().__init__(name, hp, mp, strength, defense, magic, speed, level)
        self.job = job
        self.exp = exp
        self.next_level_exp = next_level_exp
        self.skills = []  # スキルリスト（後で実装）
        # インベントリと装備品の追加
        self.inventory = {}  # アイテムIDをキー、個数を値とする辞書
        self.equipment = {
            "weapon": None,
            "armor": None,
            "accessory": None
        }    
    def gain_exp(self, amount: int) -> bool:
        """経験値を獲得する
        
        Args:
            amount: 獲得する経験値
            
        Returns:
            レベルアップしたかどうか
        """
        self.exp += amount
        
        if self.exp >= self.next_level_exp:
            return self.level_up()
        return False
    
    def level_up(self) -> bool:
        """レベルアップする
        
        Returns:
            レベルアップに成功したかどうか
        """
        self.level += 1
        
        # ステータス上昇（職業によって異なる成長率）
        if self.job == "戦士":
            self.max_hp += 25
            self.max_mp += 5
            self.strength += 3
            self.defense += 2
            self.magic += 1
            self.speed += 1
        elif self.job == "魔法使い":
            self.max_hp += 10
            self.max_mp += 15
            self.strength += 1
            self.defense += 1
            self.magic += 3
            self.speed += 1
        elif self.job == "盗賊":
            self.max_hp += 15
            self.max_mp += 10
            self.strength += 2
            self.defense += 1
            self.magic += 1
            self.speed += 3
        else:  # デフォルトの成長率
            self.max_hp += 15
            self.max_mp += 10
            self.strength += 2
            self.defense += 2
            self.magic += 2
            self.speed += 2
        
        # 次のレベルに必要な経験値を計算（レベルが上がるほど必要な経験値が増加）
        self.next_level_exp = int(self.next_level_exp * 1.5)
        
        # HPとMPを全回復
        self.current_hp = self.max_hp
        self.current_mp = self.max_mp
        
        return True
    
    def status(self) -> Dict:
        """プレイヤーキャラクターの現在の状態を取得する
        
        Returns:
            プレイヤーキャラクターの状態を表す辞書
        """
        base_status = super().status()
        
        # プレイヤー固有の情報を追加
        base_status.update({
            'job': self.job,
            'exp': self.exp,
            'next_level_exp': self.next_level_exp,
            'exp_to_next': self.next_level_exp - self.exp
        })
        
        return base_status
    def add_item(self, item_id: str, count: int = 1) -> bool:
        """アイテムをインベントリに追加する
        
        Args:
            item_id: アイテムID
            count: 追加する個数
            
        Returns:
            追加に成功したかどうか
        """
        if item_id in self.inventory:
            self.inventory[item_id] += count
        else:
            self.inventory[item_id] = count
        return True
    
    def remove_item(self, item_id: str, count: int = 1) -> bool:
        """アイテムをインベントリから削除する
        
        Args:
            item_id: アイテムID
            count: 削除する個数
            
        Returns:
            削除に成功したかどうか
        """
        if item_id not in self.inventory or self.inventory[item_id] < count:
            return False
        
        self.inventory[item_id] -= count
        if self.inventory[item_id] <= 0:
            del self.inventory[item_id]
        return True
    
    def use_item(self, item_id: str, item_dict: Dict[str, Any], target=None) -> Dict[str, Any]:
        """アイテムを使用する
        
        Args:
            item_id: 使用するアイテムID
            item_dict: アイテムIDをキー、Itemオブジェクトを値とする辞書
            target: 使用対象（未指定の場合は自分自身）
            
        Returns:
            使用結果の情報を含む辞書
        """
        # アイテムを持っていない場合
        if item_id not in self.inventory or self.inventory[item_id] <= 0:
            return {
                "success": False,
                "message": f"{item_id}を持っていません。"
            }
        
        # アイテム情報が見つからない場合
        if item_id not in item_dict:
            return {
                "success": False,
                "message": f"不明なアイテム{item_id}です。"
            }
        
        item = item_dict[item_id]
        
        # 装備品の場合
        if item.item_type == "equipment":
            result = self.equip_item(item_id, item_dict)
            return result
        
        # 消費アイテムの場合
        if target is None:
            target = self
            
        result = item.use(self, target)
        
        # 使用に成功したら1つ消費
        if result["success"]:
            self.remove_item(item_id, 1)
            
        return result

    def equip_item(self, item_id: str, item_dict: Dict[str, Any]) -> Dict[str, Any]:
        """装備品を装備する
        
        Args:
            item_id: 装備するアイテムID
            item_dict: アイテムIDをキー、Itemオブジェクトを値とする辞書
            
        Returns:
            装備結果の情報を含む辞書
        """
        # アイテムを持っていない場合
        if item_id not in self.inventory or self.inventory[item_id] <= 0:
            return {
                "success": False,
                "message": f"{item_id}を持っていません。"
            }
        
        # アイテム情報が見つからない場合
        if item_id not in item_dict:
            return {
                "success": False,
                "message": f"不明なアイテム{item_id}です。"
            }
        
        item = item_dict[item_id]
        
        # 装備品でない場合
        if item.item_type != "equipment":
            return {
                "success": False,
                "message": f"{item.name}は装備できません。"
            }
        
        # 現在装備しているアイテムを取り外す
        old_item_id = self.equipment[item.slot]
        if old_item_id:
            # インベントリに戻す
            self.add_item(old_item_id, 1)
            # 装備ボーナスを削除
            old_item = item_dict[old_item_id]
            for stat, bonus in old_item.stat_bonuses.items():
                if stat == "strength":
                    self.strength -= bonus
                elif stat == "defense":
                    self.defense -= bonus
                elif stat == "magic":
                    self.magic -= bonus
                elif stat == "speed":
                    self.speed -= bonus
                elif stat == "max_hp":
                    self.max_hp -= bonus
                elif stat == "max_mp":
                    self.max_mp -= bonus
        
        # 新しいアイテムを装備する
        self.equipment[item.slot] = item_id
        # インベントリから1つ消費
        self.remove_item(item_id, 1)
        
        # 装備ボーナスを適用
        for stat, bonus in item.stat_bonuses.items():
            if stat == "strength":
                self.strength += bonus
            elif stat == "defense":
                self.defense += bonus
            elif stat == "magic":
                self.magic += bonus
            elif stat == "speed":
                self.speed += bonus
            elif stat == "max_hp":
                self.max_hp += bonus
            elif stat == "max_mp":
                self.max_mp += bonus
        
        return {
            "success": True,
            "message": f"{item.name}を装備した！",
            "unequipped_item": old_item_id
        }

    def get_equipment_info(self, item_dict: Dict[str, Any]) -> Dict[str, str]:
        """装備情報を取得する
        
        Args:
            item_dict: アイテムIDをキー、Itemオブジェクトを値とする辞書
            
        Returns:
            スロットをキー、装備名を値とする辞書
        """
        result = {}
        for slot, item_id in self.equipment.items():
            if item_id and item_id in item_dict:
                result[slot] = item_dict[item_id].name
            else:
                result[slot] = "なし"
        return result
# ## 敵キャラクタークラス

class Enemy(Character):
    """敵キャラクター用のクラス"""
    
    def __init__(self, name: str, hp: int, mp: int, strength: int, defense: int,
                 magic: int, speed: int, level: int = 1, exp_reward: int = 10,
                 enemy_type: str = "通常"):
        """敵キャラクターの初期化
        
        Args:
            name: キャラクター名
            hp: 最大HP
            mp: 最大MP
            strength: 攻撃力
            defense: 防御力
            magic: 魔法攻撃力
            speed: 素早さ
            level: レベル（デフォルトは1）
            exp_reward: 倒した時の経験値報酬（デフォルトは10）
            enemy_type: 敵タイプ（デフォルトは「通常」）
        """
        super().__init__(name, hp, mp, strength, defense, magic, speed, level)
        self.exp_reward = exp_reward
        self.enemy_type = enemy_type
        
    def get_action(self, targets: List[Character]) -> Dict:
        """AIによる行動選択
        
        Args:
            targets: 行動対象の候補となるキャラクターリスト
            
        Returns:
            行動内容を表す辞書
        """
        # 生存しているターゲットだけをフィルタリング
        alive_targets = [t for t in targets if t.is_alive]
        
        if not alive_targets:
            return {'type': 'none', 'target': None}
        
        # シンプルなAI: ランダムに生存している対象に攻撃
        target = random.choice(alive_targets)
        
        return {
            'type': 'attack',
            'target': target
        }
    
    def status(self) -> Dict:
        """敵キャラクターの現在の状態を取得する
        
        Returns:
            敵キャラクターの状態を表す辞書
        """
        base_status = super().status()
        
        # 敵固有の情報を追加
        base_status.update({
            'enemy_type': self.enemy_type,
            'exp_reward': self.exp_reward
        })
        
        return base_status
    
 
# ## JSONからのキャラクターデータ読み込み

def load_characters_from_json(file_path: str) -> List[PlayerCharacter]:
    """JSONファイルからプレイヤーキャラクターデータを読み込む
    
    Args:
        file_path: JSONファイルのパス
        
    Returns:
        PlayerCharacterオブジェクトのリスト
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        characters = []
        for char_data in data:
            characters.append(
                PlayerCharacter(
                    name=char_data['name'],
                    hp=char_data['hp'],
                    mp=char_data['mp'],
                    strength=char_data['strength'],
                    defense=char_data['defense'],
                    magic=char_data['magic'],
                    speed=char_data['speed'],
                    level=char_data.get('level', 1),
                    job=char_data.get('job', '戦士'),
                    exp=char_data.get('exp', 0),
                    next_level_exp=char_data.get('next_level_exp', 100)
                )
            )
        return characters
    except FileNotFoundError:
        print(f"エラー: {file_path} が見つかりません。")
        return []
    except json.JSONDecodeError:
        print(f"エラー: {file_path} の形式が正しくありません。")
        return []
    except Exception as e:
        print(f"エラー: キャラクターデータの読み込み中に問題が発生しました: {e}")
        return []

def load_enemies_from_json(file_path: str) -> List[Enemy]:
    """JSONファイルから敵キャラクターデータを読み込む
    
    Args:
        file_path: JSONファイルのパス
        
    Returns:
        Enemyオブジェクトのリスト
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        enemies = []
        for enemy_data in data:
            enemies.append(
                Enemy(
                    name=enemy_data['name'],
                    hp=enemy_data['hp'],
                    mp=enemy_data['mp'],
                    strength=enemy_data['strength'],
                    defense=enemy_data['defense'],
                    magic=enemy_data['magic'],
                    speed=enemy_data['speed'],
                    level=enemy_data.get('level', 1),
                    exp_reward=enemy_data.get('exp_reward', 10),
                    enemy_type=enemy_data.get('enemy_type', '通常')
                )
            )
        return enemies
    except FileNotFoundError:
        print(f"エラー: {file_path} が見つかりません。")
        return []
    except json.JSONDecodeError:
        print(f"エラー: {file_path} の形式が正しくありません。")
        return []
    except Exception as e:
        print(f"エラー: 敵データの読み込み中に問題が発生しました: {e}")
        return []

# ## サンプルキャラクター作成とテスト

def create_sample_character() -> PlayerCharacter:
    """サンプルのプレイヤーキャラクターを作成"""
    return PlayerCharacter(
        name="勇者",
        hp=100,
        mp=50,
        strength=15,
        defense=10,
        magic=8,
        speed=12,
        job="戦士"
    )

def create_sample_enemy() -> Enemy:
    """サンプルの敵キャラクターを作成"""
    return Enemy(
        name="スライム",
        hp=50,
        mp=20,
        strength=8,
        defense=5,
        magic=3,
        speed=6,
        exp_reward=15
    )
# PlayerCharacterクラスに追加するメソッド
# __init__メソッドを書き換えるか、以下のメソッドを追加します

# # サンプルキャラクターのテスト
# if __name__ == "__main__":
#     # サンプルキャラクターのテスト
#     hero = create_sample_character()
#     print(f"勇者のステータス:")
#     for key, value in hero.status().items():
#         print(f"{key}: {value}")
    
#     # サンプル敵のテスト
#     slime = create_sample_enemy()
#     print(f"\n敵のステータス:")
#     for key, value in slime.status().items():
#         print(f"{key}: {value}")
    
#     # 基本的な戦闘テスト
#     print(f"\n簡易戦闘テスト:")
#     print(f"{hero.name}が{slime.name}に攻撃！")
#     damage = hero.attack(slime)
#     print(f"{damage}のダメージを与えた！")
#     print(f"{slime.name}のHP: {slime.current_hp}/{slime.max_hp}")
    
#     print(f"\n{slime.name}が{hero.name}に攻撃！")
#     damage = slime.attack(hero)
#     print(f"{damage}のダメージを与えた！")
#     print(f"{hero.name}のHP: {hero.current_hp}/{hero.max_hp}")